#  Weather Patterns with Meteostat

Selecting a specific location on a map, choosing a date range, downloading the data for that location, selecting variables to visualize, and then plotting and saving this data

The following Python code imports several libraries necessary for interactive data visualization and analysis:

### ipywidgets
- The `ipywidgets` library provides interactive user interface (UI) elements, such as buttons and dropdowns, for Jupyter notebooks.

### ipyleaflet
- The `ipyleaflet` library enables the creation of interactive maps within Jupyter notebooks, allowing for geographical data visualization and interaction.

### matplotlib.pyplot
- The `matplotlib.pyplot` module, often imported as `plt`, is a plotting library used to create 2D graphs and visualizations.

### seaborn
- The `seaborn` library enhances the visual aesthetics of plots created with `matplotlib`. It provides a high-level interface for creating attractive statistical graphics.

### pandas
- The `pandas` library is a powerful data manipulation and analysis tool, particularly suited for working with structured data such as tabular data.

### meteostat
- The `meteostat` library provides access to historical weather and climate data from various sources. It allows users to retrieve and analyze weather data such as temperature, precipitation, and wind speed.

## [About Meteostat](https://dev.meteostat.net/python/#installation)
The Meteostat library is a Python package that provides access to historical weather and climate data from various sources. It allows users to retrieve and analyze weather data such as temperature, precipitation, wind speed, and more for specific locations and time periods.
### Interpolation Methods

When working with point data in environmental analysis, it's often necessary to fill in missing information to create more complete time series. Meteostat offers interpolation methods to accomplish this task, leveraging data from multiple weather stations.

### Overview

Meteostat interpolates data based on the geographical distance between different weather stations and the reference point of the query. Additionally, it adjusts measurements according to differences in altitude. By projecting existing observations from weather stations onto the specified location, Meteostat provides interpolated data for precise analysis.

### Nearest Neighbor

Nearest neighbor interpolation is a straightforward method that selects the closest weather station for each record and combines all available data into a unified DataFrame. Meteostat uses this interpolation method by default.

### Weighted Average

An alternative approach is weighted average interpolation. This method considers the geographical similarity between nearby weather stations and the specified point. It assigns weights to all available data based on their proximity and aggregates them to produce an interpolated output.



In [2]:
import ipywidgets as widgets
from ipyleaflet import Map, Marker, basemaps
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from meteostat import Point, Daily
import os

# Apply Seaborn Theme For Plots

In [3]:
sns.set_theme(style="darkgrid")

# Instructions Widget

In [4]:
instructions = widgets.Label(value="1. Select a location on the map. 2. Choose start and end dates. 3. Download the data. 4. Select variables to visualize. 5. Visualize and save the data.")


# Global Variables fo latitude, longitude, and downloaded data

In [4]:
global selected_latitude, selected_longitude, downloaded_data
selected_latitude = None
selected_longitude = None
downloaded_data = pd.DataFrame()

Each key in the dictionary represents a variable, while the corresponding value represents its unit of measurement.

# Variable units dictionary

In [5]:
variable_units = {
    'time': 'Date',
    'tavg': 'Average Temperature (°C)',
    'tmin': 'Minimum Temperature (°C)',
    'tmax': 'Maximum Temperature (°C)',
    'prcp': 'Precipitation (mm)',
    'snow': 'Snow Depth (mm)',
    'wdir': 'Wind Direction (°)',
    'wspd': 'Wind Speed (km/h)',
    'wpgt': 'Peak Wind Gust (km/h)',
    'pres': 'Air Pressure (hPa)',
    'tsun': 'Sunshine Duration (m)'
}

# Initialize map for location selection

In [6]:
m = Map(center=(20, 0), zoom=2, basemap=basemaps.OpenStreetMap.Mapnik, scroll_wheel_zoom=True)
current_marker = None

def handle_map_click(**kwargs):
    global selected_latitude, selected_longitude, current_marker
    if kwargs.get('type') == 'click':
        latlon = kwargs.get('coordinates')
        selected_latitude, selected_longitude = latlon[0], latlon[1]
        if current_marker is not None:
            m.remove_layer(current_marker)
        current_marker = Marker(location=latlon)
        m.add_layer(current_marker)A

# Connect map click event
m.on_interaction(handle_map_click)

In [7]:
# Date selection widgets
start_date_picker = widgets.DatePicker(description='Start Date', disabled=False)
end_date_picker = widgets.DatePicker(description='End Date', disabled=False)

# Dropdown for selecting columns to visualize
column_selector = widgets.SelectMultiple(options=[], description='Columns:', disabled=False)

# Button to download data based on selected dates and location
download_data_button = widgets.Button(description='Download Data', button_style='info', icon='download')

# Button for visualization and saving of selected data
plot_save_button = widgets.Button(description='Visualize and Save', icon='chart-line')

# Output area
output_plot = widgets.Output()

In [8]:
def on_download_data_button_clicked(b):
    global downloaded_data
    with output_plot:
        output_plot.clear_output()
        if start_date_picker.value and end_date_picker.value and selected_latitude and selected_longitude:
            start_date = pd.to_datetime(start_date_picker.value)
            end_date = pd.to_datetime(end_date_picker.value)
            location = Point(selected_latitude, selected_longitude)
            downloaded_data = Daily(location, start_date, end_date).fetch()
            column_selector.options = downloaded_data.columns
            print("Data downloaded successfully. Please select columns to visualize and save.")
        else:
            print("Please select start and end dates, and a location on the map.")

In [9]:
def on_plot_save_button_clicked(b):
    selected_columns = column_selector.value
    if not downloaded_data.empty and selected_columns:
        with output_plot:
            output_plot.clear_output(wait=True)
            plt.figure(figsize=(10, 6))
            for column in selected_columns:
              plt.plot(downloaded_data.index, downloaded_data[column], marker='o', linestyle='-', markersize=5, label=f"{column} ({variable_units.get(column, '')})")
            plt.title('Downloaded Data Over Selected Period')
            plt.xlabel('Date')
            ylabel = 'Various Units' if len(set([variable_units[col] for col in selected_columns])) > 1 else variable_units[selected_columns[0]]
            plt.ylabel(ylabel)
            plt.xticks(rotation=45)
            plt.legend()
            plt.tight_layout()
            plt.show()
            # Ensure the directory exists
            save_dir = os.path.join('../Data')
            os.makedirs(save_dir, exist_ok=True)

            # Save selected columns to CSV
            start_str = start_date_picker.value.strftime('%Y-%m-%d')
            end_str = end_date_picker.value.strftime('%Y-%m-%d')
            columns_str = '_'.join(selected_columns).replace('/', '_')
            filename = os.path.join(save_dir, f"{columns_str}_{start_str}_to_{end_str}.csv")
            downloaded_data[list(selected_columns)].to_csv(filename)
            print(f"Data saved to '{filename}'.")
    else:
        with output_plot:
            print("No data available for visualization. Please download data first and select columns.")


# Buttons & Display widgets and instructions

In [ ]:
# Buttons
download_data_button.on_click(on_download_data_button_clicked)
plot_save_button.on_click(on_plot_save_button_clicked)
# Display widgets and instructions
display(instructions)
display(m)
widgets.VBox([widgets.HBox([start_date_picker, end_date_picker]), download_data_button, column_selector, plot_save_button, output_plot])


# 👨‍💻 Developer

This Project Was Developed And Built By Punjaji Karhale🚀